In [ ]:
# ANÁLISIS DIGITAL

import requests, time, os, random, re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from hashlib import md5
import pandas as pd

# ===== PALABRAS CLAVE DE PRUEBA =====
PALABRAS_CLAVE = ["lauritto", "paraná"]

# ===== CONFIGURACIÓN GENERAL =====
MEDIO = "analisisdigital"
MAX_PAGINAS = 3
HEADERS = {"User-Agent": "Mozilla/5.0"}
URL_BASE = "https://www.analisisdigital.com.ar/busqueda?search_api_fulltext={}&page={}"
SLEEP = (1.0, 2.5)

OUT_PATH = r"C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/pruebas/analisisdigital_lupita.xlsx"
FECHA_CORTE_DT = datetime.today() - timedelta(days=7)

# ===== PARSEO DE FECHAS =====
def parse_fecha(texto):
    meses = {
        'enero':'01','febrero':'02','marzo':'03','abril':'04','mayo':'05','junio':'06',
        'julio':'07','agosto':'08','septiembre':'09','octubre':'10','noviembre':'11','diciembre':'12'
    }
    try:
        m = re.search(r'(\d{1,2}) de ([a-záéíóú]+) de (\d{4})', texto.lower())
        if m:
            d, mes, y = int(m[1]), meses[m[2]], m[3]
            return datetime.strptime(f"{y}-{mes}-{d:02d}", "%Y-%m-%d")
    except:
        return None

# ===== SCRAPEAR UNA NOTA =====
def scrap_nota(url):
    try:
        res = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(res.text, "html.parser")

        titulo = soup.find("h1").text.strip() if soup.find("h1") else ""

        fecha_tag = soup.find('div', class_=lambda x: x and 'field--name-node-post-date' in x)
        fecha = parse_fecha(fecha_tag.get_text(strip=True)) if fecha_tag else None
        if not fecha or fecha < FECHA_CORTE_DT:
            return None

        cuerpo = soup.find('div', class_=lambda x: x and 'body-noticia' in x)
        parrafos = [p.get_text(strip=True) for p in cuerpo.find_all("p")] if cuerpo else []
        texto = "\n".join(parrafos)

        return {
            "id": md5((titulo + url).encode()).hexdigest(),
            "medio": MEDIO,
            "fecha": fecha.strftime("%Y-%m-%d"),
            "titulo": titulo,
            "contenido": texto,
            "enlace": url
        }
    except Exception as e:
        print(f"Error scrapeando nota: {url} - {e}")
        return None

# ===== BUSCAR POR PALABRA CLAVE =====
def buscar_por_palabra(palabra):
    encontrados = []
    print(f"Buscando: {palabra}")
    for pag in range(0, MAX_PAGINAS):
        url = URL_BASE.format(palabra.replace(" ", "+"), pag)
        print(f"Página {pag+1}: {url}")
        try:
            res = requests.get(url, headers=HEADERS)
            soup = BeautifulSoup(res.text, "html.parser")
            resultados = soup.find_all("div", class_="views-row")
            if not resultados:
                break

            for r in resultados:
                a = r.find("a", href=True)
                if not a:
                    continue
                link = "https://www.analisisdigital.com.ar" + a["href"]
                nota = scrap_nota(link)
                if nota:
                    nota["palabra_clave"] = palabra
                    encontrados.append(nota)

            time.sleep(random.uniform(*SLEEP))
        except Exception as e:
            print(f"Error en búsqueda de página {pag+1} para {palabra}: {e}")
            continue
    return encontrados

# ===== EJECUCIÓN PRINCIPAL =====
notas_totales = []
for palabra in PALABRAS_CLAVE:
    notas = buscar_por_palabra(palabra)
    notas_totales.extend(notas)

# ===== GUARDADO =====
if notas_totales:
    df = pd.DataFrame(notas_totales)
    df = df.drop_duplicates(subset="id").reset_index(drop=True)
    os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
    df.to_excel(OUT_PATH, index=False)
    print(f"Guardado: {len(df)} noticias en {OUT_PATH}")
else:
    print("No se encontraron noticias.")


Buscando: lauritto
Página 1: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=lauritto&page=0
Página 2: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=lauritto&page=1
Página 3: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=lauritto&page=2
Buscando: paraná
Página 1: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=paraná&page=0
Página 2: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=paraná&page=1
Página 3: https://www.analisisdigital.com.ar/busqueda?search_api_fulltext=paraná&page=2
Guardado: 76 noticias en C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/pruebas/analisisdigital_lupita.xlsx


In [ ]:
# APF DIGITAL

import os, re, time
from datetime import datetime, timedelta
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIG ===
PALABRAS_CLAVE = ["lauritto", "romero"]
BASE_URL = "https://www.apfdigital.com.ar"
BUSQUEDA_URL = BASE_URL + "/buscador/?s={}"
HEADERS = {"User-Agent": "Mozilla/5.0"}
FECHA_CORTE_DT = datetime.today() - timedelta(days=180)
OUT_PATH = r"C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/pruebas/apfdigital_debug.xlsx"

# === DRIVER ===
def setup_driver():
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# === PARSE FECHA ===
def parse_fecha_apf(txt):
    meses = {
        "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
        "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
    }
    m = re.search(r'(\d{1,2})\s+de\s+([a-záéíóú]+)\s+de\s+(\d{4})', txt.lower())
    if not m: return None
    d, mes_str, y = int(m[1]), m[2], int(m[3])
    mes = meses.get(mes_str.strip())
    if not mes: return None
    try:
        return datetime(y, mes, d)
    except:
        return None

# === SCRAPEAR DETALLE ===
def scrap_detalle(url, fallback_title):
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        titulo = soup.select_one("h1.titulo-nota")
        titulo = titulo.get_text(strip=True) if titulo else fallback_title

        copete = soup.select_one("h2.bajada")
        copete = copete.get_text(strip=True) if copete else ""

        fecha = None
        fecha_data = soup.select_one("[data-fecha]")
        if fecha_data and fecha_data.get("data-fecha"):
            try:
                fecha = datetime.strptime(fecha_data["data-fecha"], "%Y/%m/%d %H:%M:%S")
            except:
                pass
        if not fecha:
            fecha_txt = soup.select_one("div[class*='fecha'], span[class*='fecha'], time")
            if fecha_txt:
                fecha = parse_fecha_apf(fecha_txt.get_text(strip=True))

        cuerpo = (
            soup.select_one("div#cuerpo-nota")
            or soup.select_one("div.texto")
            or soup.select_one("div.noticia-contenido")
        )
        texto = " ".join(p.get_text(strip=True) for p in cuerpo.find_all(["p", "h3"])) if cuerpo else ""
        contenido = (copete + ". " + texto).strip() if copete else texto.strip()

        if not contenido:
            print("⚠️ Contenido vacío:", url)
        else:
            print("👉 Contenido truncado:", contenido[:200])

        return {
            "fecha": fecha.strftime("%Y-%m-%d") if fecha else None,
            "titulo": titulo,
            "copete": copete,
            "contenido": contenido,
            "enlace": url
        }, fecha
    except Exception as e:
        print(f"Error detalle {url}: {e}")
        return None, None

# === BUSCAR Y EXTRAER ===
def buscar_y_extraer(palabra):
    driver = setup_driver()
    encontrados = []
    try:
        url = BUSQUEDA_URL.format(quote_plus(palabra))
        driver.get(url)
        time.sleep(2)
        print(f"\n🔍 Buscando: {palabra}")

        while True:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.select("article.listado-noticias-relacionadas")
            if not cards:
                print("No se encontraron resultados.")
                break

            for card in cards:
                h2 = card.select_one("h2.text-noticia-simple-titulo")
                a = card.select_one("a[href]")
                if not h2 or not a:
                    continue
                titulo = h2.get_text(strip=True)
                href = BASE_URL + a["href"]

                print(f"\n📝 Listado: {titulo}")
                row, fecha = scrap_detalle(href, titulo)

                if not row or not fecha or fecha < FECHA_CORTE_DT:
                    continue

                row["palabra_clave"] = palabra
                encontrados.append(row)

            try:
                btn = driver.find_element(By.XPATH, "//a[contains(translate(.,'VER MÁS','ver más'),'ver más')]")
                driver.execute_script("arguments[0].scrollIntoView();", btn)
                time.sleep(0.5)
                btn.click()
                time.sleep(2)
            except NoSuchElementException:
                print("🛑 Fin del scroll.")
                break
    finally:
        driver.quit()
    return encontrados

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    todos = []
    for palabra in PALABRAS_CLAVE:
        notas = buscar_y_extraer(palabra)
        todos.extend(notas)

    if todos:
        df = pd.DataFrame(todos).drop_duplicates(subset=["enlace"]).reset_index(drop=True)
        os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
        df.to_excel(OUT_PATH, index=False)
        print(f"\n✅ Guardado en: {OUT_PATH} - Total: {len(df)} noticias.")
    else:
        print("\n⚠️ No se encontraron noticias.")



🔍 Buscando: lauritto

📝 Listado: La diputada Gaillard se reunió con Lauritto: realizará gestiones por el Rulo de la Ruta 39
👉 Contenido truncado: “Vamos a presentar un proyecto de Resolución a nivel nacional en el Congreso y vamos a hacer las gestiones ante Vialidad Nacional para que sea incluido el Rulo”, sostuvo la diputada nacional. Estas ge

📝 Listado: Lauritto se reunió con referentes de Iglesias Evangélicas: buscan una articulación para abordar problemáticas sociales
👉 Contenido truncado: El encuentro dio continuidad a la reunión celebrada en octubre del año pasado y reafirmó el compromiso de superar los modelos asistencialistas tradicionales, apostando por soluciones sostenibles desde

📝 Listado: Frigerio y Lauritto trabajaron sobre nuevos desafíos y proyectos para Concepción del Uruguay
👉 Contenido truncado: Según el intendente, el objetivo principal de la reunión fue explorar cómo la provincia puede asistir a los municipios ante este nuevo contexto. "Eso nos obliga a readecua

In [14]:
# ELONCE

import os, re, time
from datetime import datetime
from urllib.parse import urljoin, quote_plus
import pandas as pd
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIG ===
PALABRAS_CLAVE = ["lauritto", "romero"]
BASE_URL = "https://www.elonce.com"
HEADERS = {"User-Agent": "Mozilla/5.0"}
OUT_PATH = r"C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/pruebas/elonce_debug.xlsx"

# === SETUP DRIVER ===
def setup_driver():
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# === PARSE DETALLE ===
def scrap_detalle(url, fallback_title):
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        titulo = soup.select_one("h1.titulo-nota") or soup.select_one("h1")
        titulo = titulo.get_text(strip=True) if titulo else fallback_title

        copete_tag = soup.select_one("h2.bajada")
        copete = copete_tag.get_text(strip=True) if copete_tag else ""

        fecha_tag = soup.select_one("span.fecha-nota")
        fecha_str = fecha_tag.get_text(strip=True) if fecha_tag else ""
        fecha_dt = None
        m = re.search(r"(\d{1,2})\s+de\s+([a-záéíóú]+)\s+de\s+(\d{4})", fecha_str.lower())
        if m:
            meses = {
                "enero":1,"febrero":2,"marzo":3,"abril":4,"mayo":5,"junio":6,
                "julio":7,"agosto":8,"septiembre":9,"octubre":10,"noviembre":11,"diciembre":12
            }
            d, mes_str, y = int(m[1]), m[2], int(m[3])
            mes = meses.get(mes_str.strip())
            if mes:
                try: fecha_dt = datetime(y, mes, d)
                except: pass

        cuerpo_div = (
            soup.select_one("div.texto") or
            soup.select_one("div.noticia-contenido") or
            soup.select_one("div.cuerpo-nota")
        )
        texto = " ".join(p.get_text(strip=True) for p in cuerpo_div.find_all(["p", "h3"])) if cuerpo_div else ""
        contenido = (copete + ". " + texto).strip() if copete else texto.strip()

        if not contenido:
            print("⚠️ Contenido vacío:", url)
        else:
            print("👉 Contenido truncado:", contenido[:200])

        return {
            "fecha": fecha_dt.strftime("%Y-%m-%d") if fecha_dt else None,
            "titulo": titulo,
            "copete": copete,
            "contenido": contenido,
            "enlace": url
        }
    except Exception as e:
        print(f"Error detalle {url}: {e}")
        return None

# === BUSCAR Y EXTRAER ===
def buscar_y_extraer(palabra):
    encontrados = []
    drv = setup_driver()
    try:
        url = f"{BASE_URL}/buscador/?q={quote_plus(palabra)}&enviar=Buscar&ord=desc"
        drv.get(url)
        time.sleep(2)
        print(f"\n🔍 Buscando: {palabra}")

        while True:
            soup = BeautifulSoup(drv.page_source, "html.parser")
            articulos = soup.select("article.en-bandera--listado")
            if not articulos:
                print("No se encontraron artículos.")
                break

            for art in articulos:
                enlace_tag = art.select_one("a.en-bandera__ancla-title")
                if not enlace_tag:
                    continue
                href = enlace_tag.get("href")
                titulo = enlace_tag.get_text(strip=True)
                url_abs = urljoin(BASE_URL, href)

                print(f"\n📝 Listado: {titulo}")
                row = scrap_detalle(url_abs, titulo)
                if not row: continue

                row["palabra_clave"] = palabra
                encontrados.append(row)

            try:
                btn = drv.find_element(By.CLASS_NAME, "ver-mas")
                drv.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});", btn)
                time.sleep(1)
                drv.execute_script("arguments[0].click();", btn)
                time.sleep(2)
            except NoSuchElementException:
                print("🛑 Fin scroll.")
                break
    finally:
        drv.quit()
    return encontrados

# === MAIN ===
if __name__ == "__main__":
    todos = []
    for palabra in PALABRAS_CLAVE:
        notas = buscar_y_extraer(palabra)
        todos.extend(notas)

    if todos:
        df = pd.DataFrame(todos).drop_duplicates(subset=["enlace"]).reset_index(drop=True)
        os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
        df.to_excel(OUT_PATH, index=False)
        print(f"\n✅ Guardado en: {OUT_PATH} - Total: {len(df)} noticias.")
    else:
        print("\n⚠️ No se encontraron noticias.")


KeyboardInterrupt: 

In [ ]:
# Scraper UNO Digital - Prueba con buscador interno
# ======================================
# Busca por palabra clave en:
# https://www.unoentrerios.com.ar/contenidos/resultado.html?search=PALABRA
# No usa Selenium. Páginas 1 a 3.
# ======================================

import os
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote_plus
from datetime import datetime

# === CONFIG ===
PALABRAS_CLAVE = ["lauritto", "romero"]
BASE_URL = "https://www.unoentrerios.com.ar"
OUT_PATH = r"C:/Users/Lenovo/Documents/github/seguimiento-de-noticias/data/pruebas/unodigital_test.xlsx"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# === PARSEADOR DE NOTA ===
def scrap_detalle(url, fallback_title):
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        titulo = soup.find("h1")
        titulo = titulo.get_text(strip=True) if titulo else fallback_title

        copete_tag = soup.find("h2")
        copete = copete_tag.get_text(strip=True) if copete_tag else ""

        contenido_div = soup.find("div", class_="article-body") or soup.find("div", class_="nota-contenido")
        contenido = " ".join(p.get_text(strip=True) for p in contenido_div.find_all("p")) if contenido_div else ""

        texto = (copete + ". " + contenido).strip() if copete else contenido

        fecha_tag = soup.select_one("span.fecha, .article-date, .nota-fecha")
        fecha_txt = fecha_tag.get_text(strip=True) if fecha_tag else ""
        fecha_dt = None
        m = re.search(r'(\d{1,2})\s+de\s+([a-záéíóú]+)\s+de\s+(\d{4})', fecha_txt.lower())
        if m:
            meses = {
                "enero":1,"febrero":2,"marzo":3,"abril":4,"mayo":5,"junio":6,
                "julio":7,"agosto":8,"septiembre":9,"octubre":10,"noviembre":11,"diciembre":12
            }
            d, mes_str, y = int(m[1]), m[2], int(m[3])
            mes = meses.get(mes_str.strip())
            if mes:
                try: fecha_dt = datetime(y, mes, d)
                except: pass

        return {
            "titulo": titulo,
            "copete": copete,
            "contenido": texto,
            "fecha": fecha_dt.strftime("%Y-%m-%d") if fecha_dt else None,
            "enlace": url,
        }
    except Exception as e:
        print(f"❌ Error detalle {url}: {e}")
        return None

# === FUNCION PRINCIPAL ===
def buscar_palabra(palabra):
    resultados = []
    for pagina in range(3):
        url = f"{BASE_URL}/contenidos/resultado.html?search={quote_plus(palabra)}&gsc.page={pagina}"
        print(f"🔍 Buscando: {palabra} - Página {pagina+1}: {url}")
        try:
            r = requests.get(url, headers=HEADERS, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")

            enlaces = soup.select("div.gsc-webResult div.gs-title a.gs-title")
            if not enlaces:
                print("⚠️ No se encontraron resultados.")
                break

            for a in enlaces:
                href = a.get("href")
                if href and BASE_URL in href:
                    titulo = a.get_text(strip=True)
                    print("📝", titulo[:90])
                    nota = scrap_detalle(href, titulo)
                    if nota:
                        nota["palabra_clave"] = palabra
                        resultados.append(nota)

            time.sleep(1.2)
        except Exception as e:
            print(f"❌ Error en búsqueda: {e}")
            continue
    return resultados

# === MAIN ===
if __name__ == "__main__":
    todas = []
    for palabra in PALABRAS_CLAVE:
        todas.extend(buscar_palabra(palabra))

    if todas:
        df = pd.DataFrame(todas).drop_duplicates(subset="enlace")
        os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)
        df.to_excel(OUT_PATH, index=False)
        print(f"\n✅ Guardado: {len(df)} noticias en {OUT_PATH}")
    else:
        print("\n⚠️ No se encontraron noticias relevantes.")


🔍 Buscando: lauritto - Página 1: https://www.unoentrerios.com.ar/contenidos/resultado.html?search=lauritto&gsc.page=0
⚠️ No se encontraron resultados.
🔍 Buscando: romero - Página 1: https://www.unoentrerios.com.ar/contenidos/resultado.html?search=romero&gsc.page=0
⚠️ No se encontraron resultados.

⚠️ No se encontraron noticias relevantes.
